In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
    obtain_SWNUNetwork_input,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
x_data = obtain_SWNUNetwork_input(
    method="umap",
    dimension=30,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=5,
    features="time_encoding",
    standardise_method=None,
    include_features_in_path=False,
)

x_data["x_data"]["path"].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 30])

## SWNU Network

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swnu_hidden_dim_sizes_and_sig_depths": swnu_hidden_dim_sizes_and_sig_depths,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "BiLSTM": True,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [11]:
size = 5

## umap

In [12]:
(
    swnu_network_umap_kfold_5,
    best_swnu_network_umap_kfold_5,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_5_kfold_best_model.csv


In [13]:
swnu_network_umap_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2989159/1104105075.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.692575   
                                                       0.0005         0.692016   
                                                       0.0010         0.686362   
                                          0.2          0.0001         0.686984   
                                                       0.0005         0.693569   
                                                       0.0010         0.689282   
                           (512, 512)     0.1          0.0001         0.688972   
                                                       0.0005         0.689034   
                                                       0.0010         0.692700   
                                          0.2          0.0001         0.691457   
                                                       0.0005         0.689655   
                                                       0.0010         0.692203   
           (12,)           (256, 256)     0.1          0.0001         0.695371   
                                                       0.0005         0.693010   
                                                       0.0010         0.684436   
                                          0.2          0.0001         0.693818   
                                                       0.0005         0.693507   
                                                       0.0010         0.685430   
                           (512, 512)     0.1          0.0001         0.690587   
                                                       0.0005         0.684623   
                                                       0.0010         0.698602   
                                          0.2          0.0001         0.693880   
                                                       0.0005         0.688910   
                                                       0.0010         0.690649   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.665617   
                                                       0.0005         0.665610   
                                                       0.0010         0.663096   
                                          0.2          0.0001         0.663260   
                                                       0.0005         0.667578   
                                                       0.0010         0.665382   
                           (512, 512)     0.1          0.0001         0.664952   
                                                       0.0005         0.664354   
                                                       0.0010         0.666706   
                                          0.2          0.0001         0.666502   
                                                       0.0005         0.666259   
                                                       0.0010         0.665859   
           (12,)           (256, 256)     0.1          0.0001         0.668192   
                                                       0.0005         0.670532   
                                                       0.0010         0.663251   
                                          0.2          0.0001         0.669444   
                                                       0.0005         0.668718   
                                                       0.0010         0.665569   
                           (512, 512)     0.1          0.0001         0.665440   
                                                       0.0005         0.663222   
                                                       0.0010         0.670245   
                                          0.2      

In [14]:
best_swnu_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.694501,0.670570,"[0.7593598590515342, 0.581781066598622]",0.668244,"[0.7782124586217274, 0.5582761998041136]",0.674376,"[0.7413990825688074, 0.607352157698455]",None,0.736487,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.685927,0.661471,"[0.7524607022183045, 0.5704817741524343]",0.659269,"[0.7716179572160289, 0.5469208211143695]",0.665198,"[0.7342316513761468, 0.5961640916355887]",None,0.735679,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.691333,0.664278,"[0.7595818815331009, 0.5689744924518481]",0.662823,"[0.7694117647058824, 0.556234096692112]",0.666156,"[0.75, 0.582312200319659]",None,0.731500,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_swnu_network_umap_kfold_5["f1"].mean()

0.6654399626676407

In [16]:
best_swnu_network_umap_kfold_5["precision"].mean()

0.6634455496923723

In [17]:
best_swnu_network_umap_kfold_5["recall"].mean()

0.6685765305997761

In [18]:
np.stack(best_swnu_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.75713415, 0.57374578])

In [19]:
np.stack(best_swnu_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.77308073, 0.55381037])

In [20]:
np.stack(best_swnu_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.74187691, 0.59527615])

# w=11

In [21]:
size = 11

## umap

In [22]:
(
    swnu_network_umap_kfold_11,
    best_swnu_network_umap_kfold_11,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_11_kfold_best_model.csv


In [23]:
swnu_network_umap_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_2989159/2463364505.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.685306   
                                                       0.0005         0.685430   
                                                       0.0010         0.693135   
                                          0.2          0.0001         0.683939   
                                                       0.0005         0.686424   
                                                       0.0010         0.690960   
                           (512, 512)     0.1          0.0001         0.691519   
                                                       0.0005         0.679404   
                                                       0.0010         0.690774   
                                          0.2          0.0001         0.694936   
                                                       0.0005         0.683380   
                                                       0.0010         0.685120   
           (12,)           (256, 256)     0.1          0.0001         0.694936   
                                                       0.0005         0.701149   
                                                       0.0010         0.687418   
                                          0.2          0.0001         0.695993   
                                                       0.0005         0.689779   
                                                       0.0010         0.691022   
                           (512, 512)     0.1          0.0001         0.696365   
                                                       0.0005         0.691954   
                                                       0.0010         0.686176   
                                          0.2          0.0001         0.697111   
                                                       0.0005         0.696800   
                                                       0.0010         0.682510   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.664227   
                                                       0.0005         0.664020   
                                                       0.0010         0.668533   
                                          0.2          0.0001         0.663859   
                                                       0.0005         0.666887   
                                                       0.0010         0.667018   
                           (512, 512)     0.1          0.0001         0.669747   
                                                       0.0005         0.663382   
                                                       0.0010         0.664446   
                                          0.2          0.0001         0.671949   
                                                       0.0005         0.664385   
                                                       0.0010         0.665520   
           (12,)           (256, 256)     0.1          0.0001         0.672894   
                                                       0.0005         0.677857   
                                                       0.0010         0.666203   
                                          0.2          0.0001         0.674500   
                                                       0.0005         0.670456   
                                                       0.0010         0.669228   
                           (512, 512)     0.1          0.0001         0.676612   
                                                       0.0005         0.671072   
                                                       0.0010         0.664224   
                                          0.2      

In [24]:
best_swnu_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.707176,0.685383,"[0.7681865132064335, 0.6025803187452567]",0.682563,"[0.7914259653390088, 0.5736994219653179]",0.690398,"[0.7462729357798165, 0.6345231752797017]",None,0.747540,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.683131,0.664164,"[0.7439759036144579, 0.5843520782396089]",0.661810,"[0.7836294416243654, 0.5399909624943515]",0.672398,"[0.7081422018348624, 0.6366542354821524]",None,0.746192,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.697670,0.673952,"[0.7618907809747505, 0.5860132720775907]",0.671578,"[0.7806859205776173, 0.5624693777560019]",0.677797,"[0.7439793577981652, 0.6116142781033564]",None,0.749427,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [25]:
best_swnu_network_umap_kfold_11["f1"].mean()

0.6744998111430164

In [26]:
best_swnu_network_umap_kfold_11["precision"].mean()

0.6719835149594439

In [27]:
best_swnu_network_umap_kfold_11["recall"].mean()

0.6801976973796758

In [28]:
np.stack(best_swnu_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.75801773, 0.59098189])

In [29]:
np.stack(best_swnu_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78524711, 0.55871992])

In [30]:
np.stack(best_swnu_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.73279817, 0.62759723])

# w=20

In [11]:
size = 20

## umap

In [12]:
(
    swnu_network_umap_kfold_20,
    best_swnu_network_umap_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_20_kfold_best_model.csv


In [13]:
swnu_network_umap_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3569902/29893126.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_20.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.684809   
                                                       0.0005         0.689034   
                                                       0.0010         0.679528   
                                          0.2          0.0001         0.677043   
                                                       0.0005         0.685492   
                                                       0.0010         0.679279   
                           (512, 512)     0.1          0.0001         0.686362   
                                                       0.0005         0.678037   
                                                       0.0010         0.677664   
                                          0.2          0.0001         0.684747   
                                                       0.0005         0.671202   
                                                       0.0010         0.682510   
           (12,)           (256, 256)     0.1          0.0001         0.693135   
                                                       0.0005         0.695433   
                                                       0.0010         0.681143   
                                          0.2          0.0001         0.692824   
                                                       0.0005         0.684436   
                                                       0.0010         0.684995   
                           (512, 512)     0.1          0.0001         0.685182   
                                                       0.0005         0.691768   
                                                       0.0010         0.679155   
                                          0.2          0.0001         0.688226   
                                                       0.0005         0.695993   
                                                       0.0010         0.684685   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.665335   
                                                       0.0005         0.668113   
                                                       0.0010         0.665703   
                                          0.2          0.0001         0.657585   
                                                       0.0005         0.665343   
                                                       0.0010         0.664253   
                           (512, 512)     0.1          0.0001         0.666271   
                                                       0.0005         0.661279   
                                                       0.0010         0.663799   
                                          0.2          0.0001         0.665427   
                                                       0.0005         0.654849   
                                                       0.0010         0.667948   
           (12,)           (256, 256)     0.1          0.0001         0.673808   
                                                       0.0005         0.676220   
                                                       0.0010         0.663267   
                                          0.2          0.0001         0.674928   
                                                       0.0005         0.669849   
                                                       0.0010         0.667831   
                           (512, 512)     0.1          0.0001         0.669580   
                                                       0.0005         0.674058   
                                                       0.0010         0.664650   
                                          0.2      

In [14]:
best_swnu_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.690028,0.675956,"[0.7434829554218726, 0.6084294796326819]",0.674911,"[0.8046744574290484, 0.5451476793248945]",0.689636,"[0.6909403669724771, 0.6883324453915823]",None,0.784472,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.692078,0.672714,"[0.7523238380809595, 0.5931034482758619]",0.670016,"[0.7884978001257071, 0.5515345854328905]",0.680386,"[0.7193233944954128, 0.6414491209376665]",None,0.824639,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.705871,0.685957,"[0.7650387135199523, 0.6068759342301944]",0.682903,"[0.7958488228004956, 0.5699578848853533]",0.692717,"[0.7365252293577982, 0.648907831646244]",None,0.795121,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_swnu_network_umap_kfold_20["f1"].mean()

0.6782090615269204

In [16]:
best_swnu_network_umap_kfold_20["precision"].mean()

0.6759435383330649

In [17]:
best_swnu_network_umap_kfold_20["recall"].mean()

0.6875797314668635

In [18]:
np.stack(best_swnu_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.75361517, 0.60280295])

In [19]:
np.stack(best_swnu_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.79634036, 0.55554672])

In [20]:
np.stack(best_swnu_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.71559633, 0.65956313])

# w=35

In [21]:
size = 35

## umap

In [ ]:
(
    swnu_network_umap_kfold_35,
    best_swnu_network_umap_kfold_35,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swnu_network_umap_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_umap_kfold_35

In [ ]:
best_swnu_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["recall_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## umap

In [ ]:
(
    swnu_network_umap_kfold_80,
    best_swnu_network_umap_kfold_80,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swnu_network_umap_kfold_80.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_umap_kfold_80

In [ ]:
best_swnu_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["recall_scores"]).mean(axis=0)